In [25]:
import pandas as pd
import folium
import html5lib
import folium
import numpy as np
import googlemaps
from datetime import datetime
import json
import goslate
import requests

# Extraction of Data from Grants csv file

In [3]:
grants = pd.read_csv('Data/P3_GrantExport.csv',sep=";")
grants.head()

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


In [85]:
grants[grants['University']=='Weitere Institute - FINST']

83       Institut für psychologische Forschung und Bera...
84       Institut für psychologische Forschung und Bera...
7436                           Centre horticole de Lullier
7491                           Centre horticole de Lullier
10434    Laboratorio di Patologia Cellulare Istituto Ca...
10714                                 CERN Theory Division
11045    Laboratorio di Patologia Cellulare Istituto Ca...
12381               Marie Meierhofer-Institut für das Kind
14098    Schweizerische Akademie für Entwicklung SAD Li...
14903               Marie Meierhofer-Institut für das Kind
15305                 Institut für Kommunikationsforschung
15604    Laboratorio di Patologia Cellulare Istituto Ca...
17054    Laboratorio di Patologia Cellulare Istituto Ca...
17663                 Institut für Kommunikationsforschung
18043               Marie Meierhofer-Institut für das Kind
20306    Laboratorio di Patologia Cellulare Istituto Ca...
20324    Laboratorio di Patologia Cellulare Istituto Ca.

We keep only two columns: University and Approved Amount

In [11]:
amount_university_grants = grants[["University","Approved Amount"]]

In [12]:
amount_university_grants["Approved Amount"].describe()

count                       63969
unique                      36455
top       data not included in P3
freq                        10910
Name: Approved Amount, dtype: object

In [13]:
len(amount_university_grants[amount_university_grants["Approved Amount"]=='data not included in P3'])

10910

For some rows, the approved amount is not included and have a value of 'data not included in P3'. We decided to remove those. 

In [14]:
grants_without_nan = amount_university_grants[amount_university_grants["Approved Amount"]!='data not included in P3']

In [15]:
grants_without_nan.describe()

,University,Approved Amount
count,50878,53059
unique,77,36454
top,Universität Zürich - ZH,10000.00
freq,6754,611


In [10]:
grants_without_nan["Approved Amount"].dtype

dtype('O')

Since the type of this column is object, we convert it into numeric values.

In [18]:
grants_without_nan.loc["Approved Amount"] = pd.to_numeric(grants_without_nan["Approved Amount"],errors='coerce')
grants_without_nan

/home/meryem/miniconda2/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,University,Approved Amount
0,Nicht zuteilbar - NA,11619.0
1,Université de Genève - GE,41022.0
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0
3,Universität Basel - BS,52627.0
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0
5,Université de Fribourg - FR,53009.0
6,Université de Fribourg - FR,25403.0
7,Universität Zürich - ZH,47100.0
8,Université de Lausanne - LA,25814.0
9,Université de Genève - GE,360000.0


In [19]:
grants_without_nan.dtypes

University          object
Approved Amount    float64
dtype: object

In [72]:
df = grants_without_nan.groupby(["University"])["Approved Amount"].sum().reset_index()

In [60]:
df.shape

(77, 2)

In [73]:
university_amount_df = pd.DataFrame(df)
university_amount_df.head()

,University,Approved Amount
0,AO Research Institute - AORI,3435621.0
1,Allergie- und Asthmaforschung - SIAF,19169965.0
2,Berner Fachhochschule - BFH,31028695.0
3,Biotechnologie Institut Thurgau - BITG,2492535.0
4,Centre de rech. sur l'environnement alpin - CR...,1567678.0


# Mapping University names to cantons

We add the component 'components=country:ch' to force the api to look only in Switzerland

In [62]:
def getCantonId(Uni):
    Go_URL='https://maps.googleapis.com/maps/api/geocode/json?components=country:ch&' # ADD YOUR KEY HERE
    r = requests.get(Go_URL, params={"address": Uni})
    json=r.json()
    i=0
    b=''
    for a in json['results'][0]['address_components']:
        if(json['results'][0]['address_components'][i]['types'][0]=='administrative_area_level_1'):
            b=json['results'][0]['address_components'][i]['short_name']
            break
        i+=1
    
    return b

In [27]:
getCantonId('eth')

'ZH'

In [74]:
canton_mapping=[]
for index, row in university_amount_df.iterrows():
    c=row['University'].split('-')[0]
    zz=getCantonId(c)
    canton_mapping.append(zz)

In [75]:
university_amount_df['Canton1'] = canton_mapping
university_amount_df.head()

,University,Approved Amount,Canton1
0,AO Research Institute - AORI,3435621.0,
1,Allergie- und Asthmaforschung - SIAF,19169965.0,
2,Berner Fachhochschule - BFH,31028695.0,
3,Biotechnologie Institut Thurgau - BITG,2492535.0,TG
4,Centre de rech. sur l'environnement alpin - CR...,1567678.0,


### Manual Mappings

In [86]:
university_amount_df['Canton1'][university_amount_df['University']=='AO Research Institute - AORI']='GR'
university_amount_df['Canton1'][university_amount_df['University']=='Allergie- und Asthmaforschung - SIAF']='GR'
university_amount_df['Canton1'][university_amount_df['University']=='Berner Fachhochschule - BFH']='BE'
university_amount_df['Canton1'][university_amount_df['University']=="Centre de rech. sur l'environnement alpin - CREALP"]='VS'
university_amount_df['Canton1'][university_amount_df['University']=="Eidg. Anstalt für Wasserversorgung - EAWAG"]='ZH'
university_amount_df['Canton1'][university_amount_df['University']=="Eidg. Hochschulinstitut für Berufsbildung - EHB"]='BE'
university_amount_df['Canton1'][university_amount_df['University']=="Ente Ospedaliero Cantonale - EOC"]='TI'
university_amount_df['Canton1'][university_amount_df['University']=="Fachhochschule Kalaidos - FHKD"]='ZH'
university_amount_df['Canton1'][university_amount_df['University']=="Fachhochschule Nordwestschweiz (ohne PH) - FHNW"]='SO'
university_amount_df['Canton1'][university_amount_df['University']=="Fachhochschule Ostschweiz - FHO"]='SG'
university_amount_df['Canton1'][university_amount_df['University']=="Zürcher Fachhochschule (ohne PH) - ZFH"]='ZH'
university_amount_df['Canton1'][university_amount_df['University']=="SUP della Svizzera italiana - SUPSI"]='TI'

university_amount_df['Canton1'][university_amount_df['University']=="Forschungskommission SAGW"]='LU'
university_amount_df['Canton1'][university_amount_df['University']=="Forschungsinstitut für Opthalmologie - IRO"]='VS'
university_amount_df['Canton1'][university_amount_df['University']=="Franklin University Switzerland - FUS"]='TI'
university_amount_df['Canton1'][university_amount_df['University']=="Physikal.-Meteorolog. Observatorium Davos - PMOD"]='GR'

university_amount_df['Canton1'][university_amount_df['University']=="Haute école pédagogique fribourgeoise - HEPFR"]='FR'

university_amount_df['Canton1'][university_amount_df['University']=="Idiap Research Institute - IDIAP"]='VS'

university_amount_df['Canton1'][university_amount_df['University']=="Inst. Suisse de Spéléologie et Karstologie - ISSKA"]='NE'
university_amount_df['Canton1'][university_amount_df['University']=="Inst. de Hautes Etudes Internat. et du Dév - IHEID"]='GE'
university_amount_df['Canton1'][university_amount_df['University']=="Inst. universit. romand de Santé au Travail - IST"]='VD'

university_amount_df['Canton1'][university_amount_df['University']=="Pädagogische Hochschule Nordwestschweiz - PHFHNW"]='AG'
university_amount_df['Canton1'][university_amount_df['University']=="Schweiz. Institut für Kunstwissenschaft - SIK-ISEA"]='ZH'

university_amount_df['Canton1'][university_amount_df['University']=="Schweizer Kompetenzzentrum Sozialwissensch. - FORS"]='VD'

university_amount_df['Canton1'][university_amount_df['University']=="Schweizer Paraplegiker Forschung - SPF"]='LU'
university_amount_df['Canton1'][university_amount_df['University']=="Swiss Center for Electronics and Microtech. - CSEM"]='NE'

university_amount_df['Canton1'][university_amount_df['University']=="Swiss Institute of Bioinformatics - SIB"]='VD'
university_amount_df['Canton1'][university_amount_df['University']=="Université de Neuchâtel - NE"]='NE'
university_amount_df['Canton1'][university_amount_df['University']==" Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHSE "]='VS'
university_amount_df['Canton1'][university_amount_df['University']=='Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS']='VS'
university_amount_df['Canton1'][university_amount_df['University']=='Forschungsanstalten Agroscope - AGS']='ZH'
university_amount_df['Canton1'][university_amount_df['University']=='Forschungsinstitut für biologischen Landbau - FIBL']='AG'
university_amount_df['Canton1'][university_amount_df['University']=='Friedrich Miescher Institute - FMI']='BS'
university_amount_df['Canton1'][university_amount_df['University']=='HES de Suisse occidentale - HES-SO']='JU'
university_amount_df['Canton1'][university_amount_df['University']=='Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP'] = 'TI'
university_amount_df['Canton1'][university_amount_df['University']=='Università della Svizzera italiana - USI']= 'TI'

/home/meryem/miniconda2/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/meryem/miniconda2/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/meryem/miniconda2/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()

In [88]:
university_amount_df

,University,Approved Amount,Canton1
0,AO Research Institute - AORI,3.435621e+06,GR
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,GR
2,Berner Fachhochschule - BFH,3.102870e+07,BE
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,TG
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,VS
5,EPF Lausanne - EPFL,1.175316e+09,VD
6,ETH Zürich - ETHZ,1.635597e+09,ZH
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,ZH
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,BE


There are three categories of universities for which we didn't succeed to find a mapping:
1. Universities provided under the name of "Other Universities" as translated from german:
   	'Weitere Institute - FINST' and 'Weitere Spitäler - ASPIT', '	NPO (Biblioth., Museen, Verwalt.) - NPO'
2. No assignable values: 'Nicht zuteilbar - NA'
3. Universities not in Switzerland: '7 Istituto Svizzero di Roma - ISR' which is in Roma,    Italy
4. Universities we found in multiple cantons: 'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL' and 'Eidg. Material und Prüfungsanstalt - EMPA'
5. Private Companies: 'Firmen/Privatwirtschaft - FP'

In [97]:
university_amount_df = university_amount_df[university_amount_df["Canton1"]!='']
university_amount_df

,University,Approved Amount,Canton1
0,AO Research Institute - AORI,3.435621e+06,GR
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,GR
2,Berner Fachhochschule - BFH,3.102870e+07,BE
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,TG
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,VS
5,EPF Lausanne - EPFL,1.175316e+09,VD
6,ETH Zürich - ETHZ,1.635597e+09,ZH
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,ZH
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,BE
11,Ente Ospedaliero Cantonale - EOC,5.067172e+06,TI


Now, we aggregate using groupby to get the sum of the amounts for each canton.

In [98]:
df = university_amount_df.groupby(["Canton1"])["Approved Amount"].sum().reset_index()

In [99]:
canton_amount_df = pd.DataFrame(df)
canton_amount_df

,Canton1,Approved Amount
0,AG,1.261875e+08
1,BE,1.554479e+09
2,BS,1.392498e+09
3,FR,4.590737e+08
4,GE,1.877102e+09
5,GR,3.653832e+07
6,JU,3.416297e+07
7,LU,5.477329e+07
8,NE,4.025250e+08
9,SG,9.119410e+07


# Getting the cantons from json file

In [102]:
# Do not forget to explain the format of the json
with open('json/ch-cantons.topojson.json') as json_data:
    cantons_data = json.load(json_data)

cantons_mappings = []
for i in range(0,len(cantons_data["objects"]["cantons"]["geometries"])):
    canton_id = cantons_data["objects"]["cantons"]["geometries"][i]["id"]
    canton_name = cantons_data["objects"]["cantons"]["geometries"][i]["properties"]["name"]
    cantons_mappings.append((canton_name,canton_id))
    
cantons_mappings_dict = dict(cantons_mappings)
cantons_mappings_dict.values()

dict_values(['SZ', 'BL', 'ZG', 'SG', 'BE', 'AR', 'GE', 'AI', 'SO', 'BS', 'UR', 'TI', 'OW', 'TG', 'JU', 'ZH', 'NE', 'NW', 'FR', 'VS', 'LU', 'GL', 'VD', 'AG', 'GR', 'SH'])

In [103]:
existing_cantons = canton_amount_df['Canton1'].values
for canton_id in cantons_mappings_dict.values():
    if (canton_id not in existing_cantons):
        canton_amount_df.loc[len(canton_amount_df)]=[canton_id,0]
        
canton_amount_df

,Canton1,Approved Amount
0,AG,1.261875e+08
1,BE,1.554479e+09
2,BS,1.392498e+09
3,FR,4.590737e+08
4,GE,1.877102e+09
5,GR,3.653832e+07
6,JU,3.416297e+07
7,LU,5.477329e+07
8,NE,4.025250e+08
9,SG,9.119410e+07


# Drawing of Choropleth Map

In [104]:
cantons_topo_path = r'json/ch-cantons.topojson.json'

swiss_map = folium.Map(location=[47.05016819999999, 8.309307200000035],
                   tiles='Mapbox Bright', zoom_start=7)
swiss_map.choropleth(geo_path=cantons_topo_path, topojson='objects.cantons')
#swiss_map.save('html/swiss_map.html')
swiss_map

# Binding of data between Pandas DataFrame and Geo/TopoJSON geometries

In [111]:
swiss_map = folium.Map(location=[47.05016819999999, 8.309307200000035],
                   tiles='Mapbox Bright', zoom_start=7)
swiss_map.choropleth(geo_path=cantons_topo_path, 
                     data=canton_amount_df,
                     columns=['Canton1', 'Approved Amount'],
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='YlGn',
                     fill_opacity=100,
                     line_opacity=1000,
                     legend_name = 'Total Amount of Grants'
                    )

swiss_map

/home/meryem/miniconda2/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
